# Doordash Earn By Offer Cherry Picking

## Overview
Doordash has two primary earning models wherein delivery drivers (aka, "Dashers") receive an amount from Doordash, called Doordash pay, and tips from the customer.  The **Earn by Time** (EBT) model sets the amount of Doordash pay based on time spent during the delivery.  The **Earn By Offer** (EBO) model provides a fixed amount of Doordash pay for the delivery.  Dashers choose between the models when starting a dash, and the behavior of the platform, rules of engagement, and the affect on citical dasher statistics and earnings are all driven by this choice. In the dasher community there are many theories about Doordash algorithms, customer behavior, independant contractor status and the sovereingty it brings, and many other narratives that all take place in the absence of hard information.
One such narrative is whether the EBT model is inferior to the EBO model.

The chief difference between the EBT and EBO models is as follows:
- EBO Dashers can see an up-front offer for a delivery and can choose to reject the offer if they don't like the amount.
- EBT Dashers do not see an offer amount in advance, and can only reject one offer per hour.

Several consequences flow from these models.  One major consequence is that EBT drivers cannot reject non-tipping customers because they lack the information and have limited rejection capabilities, and for EBO dashers, liberal ability to reject offers can lower their acceptance rate, which has consequences on the platform that include access to scheduling. Dashers have lore, and a rich social media presence, so a culture has emerged that promotes the idea of **Cherry Picking**, which means dashing in the EBO model and rejecting any offer that doesn't meet some set of criteria.  Belief attaches in the creation of this criteria.  Because information is kept deliberately sparse by Doordash, dashers create reasons for rejecting an offer.  These reasons are rife with confirmation bias, and extra effort is needed to combat this.

This experiment seeks to generate solid data to use in answering several questions about Doordash, customer and order quality, and behavior when cherrypicking offers under EBO.  The full experiment plan can be found [[here]].

## Methods
The author is a part-time dasher who usually works using the EBT model.  We will call this person "driver". To conduct this experiment, the driver dashed by EBO for five weeks, from 2024.07.29 to 2024.08.28 - a duration of 5 weeks.  The driver used specific offer rejection criteria to determine whether to accept or reject an offer.  The detail of that criteria can be found in the full experiment plan (see above). A total of 18 dashing days were recorded. The driver kept screenshots of each offer, delivery, and pay results from the Dasher mobile app.  In addition to this electronic data, the driver kept a manual log of every delivery to capture information that Doordash does not offer from the Dasher app. Doordash provides dashers with the ability to request and download what Doordash calls a complete datasaet.  The driver requested this information, which provides order, pickup and delivery times in UTC, the names of providers (as passed via integrations) and the number of order items and the dollar amount of the order. All of these sources were combined into an electronic log kept in Libreoffice Calc, with additional annotations to characterize each delivery.  That log was exported as a CSV file to include in this analysis.  The remainder of this notebook includes these analyses.




### Data Catalog
#### CherryPicking_CoreDeliveryData.csv

**Experiment.Day** The ordinal day of the dash during the experiment.

**X** The ordinal delivery number

**Date** The date of the delivery.

**Time** The time in 24-hour form of when the offer was received.  Manually recorded by the driver at minute resolution.

**Mileage.at.Offer** Odometer mileage at offer, manually recorded recorded by the driver.

**Offer** Ordinal offer for the dash, manually recorded by the driver.

**Delivery** Ordinal delivery for the dash, manually recorded by the driver.

**Offer..** Amount offered by Doordash in the Dasher app, as captured by screenshot.

**Offer.Miles** Mileage as supplied by Doordash in the Dasher app, as captured by screenshot.

**Provider** The order provider (restaurant) as manually recorded by the driver.

**Provider.city** Either Pullman, Wa or Moscow Id, the two cities that make up the driver's only available zone.

**Arrival.at.Store** Time of arrival at the provider in 24 hr format as manually recorded by the driver.

**Return.with.Order** Time the driver returned to the car with an order in 24 hr format, as manually recorded by the driver.

**Customer.Address** Delivery address, as manually recorded by the driver.

**Multi.Family.Unit** An apartment or unit number, NA for a single family home or business, as manually recorded by the driver.

**Destination City** City of the delivery address, either Pullman Wa or Moscow Id, as manually recorded by the driver.

**Arrival.at.Customer** Time of arrival at the delivery location in 24 hr format as manually recorded by the driver.

**Return.from.Dropoff** Time of return to the car from dropping off the order, in 24 hr format as manually recorded by the driver.

**DDPAY** Doordash pay amount, collected from screenshot.

**DD.Peak** Doordash promotional peak pay, when promotions were active, as taken from screenshot.

**TIP** Customer tip amount, as taken form screenshot.

**Notes** Driver notes for anomaly, other unusual circumstances.

**Offer.Type** Manual attribution of the offer type, per the following taxonomy:
- **A** Anomolous delivery - something about the delivery makes it prudent to remove it from analyis
- **C** Conventional Delivery
- **S** A single delivery in a stacked offer
- **HL** Host and Leach: A delivery in a stacked offer where Doordash does not issue any Doordash pay.
- **HL+** Host and Leach: A delivey in a stacked offer where there is neither Doordash pay nor a customer tip.

**ORDER_CREATED_TIME** Order creation time in UTC as supplied by driver's Doordash data request.

**ACTUAL_PICKUP_TIME** Actual time in UTC that the driver clicked "Confirm Pickup" in the dasher app as supplied by driver's Doordash data request.

**ACTUAL_DELIVERY_TIME** Actual time in UTC that the driver clicked "Complete Delivery" in the dasher app as supplied by driver's Doordash data request.

**STORE_NAME** Doordash provider name as supplied by driver's Doordash data request.

**TOTAL_ITEM_COUNT** Number of items in the order as supplied by driver's Doordash data request.

**SUBTOTAL_IN_CENTS** Amount of the customer's order as supplied by driver's Doordash data request.

**ORDER_STATUS** How the driver completed the order, as supplied by driver's Doordash data request.

### Setup

In [7]:
library(tidyverse)

### Source Data

In [12]:
# Set the working directory to the repository /data directory.
setwd("../data")
getwd()

[1] "/home/woodst/Documents/Data/Dropbox/Doordash/DoorDash_Analysis/data"

In [17]:
# Load the data and look at the initial structure.
cpdata=read.csv("CherryPicking_CoreDeliveryData.csv", header=TRUE)
str(cpdata)

'data.frame':	157 obs. of  30 variables:
 $ Experiment.Day      : int  1 1 1 1 1 1 1 1 1 1 ...
 $ X                   : int  1 2 3 4 5 6 7 8 9 10 ...
 $ Date                : chr  "07/29/24" "07/29/24" "07/29/24" "07/29/24" ...
 $ Time                : chr  "17:30" "18:25" "18:25" "18:53" ...
 $ Mileage.at.Offer    : int  220985 220999 220999 221004 221018 221023 221025 221027 221030 221034 ...
 $ Offer               : int  1 2 2 3 4 5 6 7 8 9 ...
 $ Delivery            : int  1 2 3 4 5 6 7 8 9 10 ...
 $ Offer..             : chr  "16.75" "9.50" "0.00" "13.50" ...
 $ Offer.Miles         : num  12 7.1 0 13.1 5.7 1.2 1.2 1.4 3.6 2.9 ...
 $ Provider            : chr  "Safeway Floral" "Sonic" "Sonic" "KFC" ...
 $ Provider.City       : chr  "Pullman" "Moscow" "Moscow" "Moscow" ...
 $ Arrival.at.Store    : chr  "17:31" "18:30" "18:30" "18:57" ...
 $ Return.With.Order   : chr  "17:46" "18:37" "18:37" "19:00" ...
 $ Customer.Address    : chr  "652 Pavel Ct" "1551 N Polk Extension" "1881 E F St

### Wrangling

In [ ]:
# Date conversion from string to date type
